# YAP 470 Project - Predict Future Sales

Name: Alp Bora Kirte ID: 191201077 Course: YAP 470/BIL 570

In [1]:
# imports
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sktime.forecasting.model_selection import SlidingWindowSplitter,temporal_train_test_split
from sktime.utils.plotting import plot_series

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.compose import make_reduction
from sktime.forecasting.model_selection import temporal_train_test_split
from sklearn.pipeline import make_pipeline
from sktime.transformations.panel.tsfresh import TSFreshFeatureExtractor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import roc_curve, auc, accuracy_score
from tqdm import tqdm

C:\Users\Alp Bora\AppData\Local\Programs\Python\Python310\lib\site-packages\sktime\utils\validation\_dependencies.py:117: UserWarning: No module named 'tsfresh'. 'tsfresh' is a soft dependency and not included in the base sktime installation. Please run: `pip install tsfresh` to install the tsfresh package. To install all soft dependencies, run: `pip install sktime[all_extras]`
  warnings.warn(msg)


In [2]:
for dirname, _, filenames in os.walk(r'C:\Users\Alp Bora\workarea\Yap470\YAP_470_Project-Predict_Future_Sales'):
    for filename in filenames:
        if filename.endswith('.csv'):
            print(os.path.join(dirname, filename))

C:\Users\Alp Bora\workarea\Yap470\YAP_470_Project-Predict_Future_Sales\items-translated.csv
C:\Users\Alp Bora\workarea\Yap470\YAP_470_Project-Predict_Future_Sales\items.csv
C:\Users\Alp Bora\workarea\Yap470\YAP_470_Project-Predict_Future_Sales\item_categories-translated.csv
C:\Users\Alp Bora\workarea\Yap470\YAP_470_Project-Predict_Future_Sales\item_categories.csv
C:\Users\Alp Bora\workarea\Yap470\YAP_470_Project-Predict_Future_Sales\sales_train.csv
C:\Users\Alp Bora\workarea\Yap470\YAP_470_Project-Predict_Future_Sales\sample_submission.csv
C:\Users\Alp Bora\workarea\Yap470\YAP_470_Project-Predict_Future_Sales\shops-translated.csv
C:\Users\Alp Bora\workarea\Yap470\YAP_470_Project-Predict_Future_Sales\shops.csv
C:\Users\Alp Bora\workarea\Yap470\YAP_470_Project-Predict_Future_Sales\test.csv


In [3]:
items_path = r'C:\Users\Alp Bora\workarea\Yap470\YAP_470_Project-Predict_Future_Sales\items-translated.csv'
item_categories_path = r'C:\Users\Alp Bora\workarea\Yap470\YAP_470_Project-Predict_Future_Sales\item_categories-translated.csv'
sales_train_path = r'C:\Users\Alp Bora\workarea\Yap470\YAP_470_Project-Predict_Future_Sales\sales_train.csv'
shops_path = r'C:\Users\Alp Bora\workarea\Yap470\YAP_470_Project-Predict_Future_Sales\shops-translated.csv'
test_path = r'C:\Users\Alp Bora\workarea\Yap470\YAP_470_Project-Predict_Future_Sales\test.csv'

items = pd.read_csv(items_path)
item_categories = pd.read_csv(item_categories_path)
sales = pd.read_csv(sales_train_path)
shops = pd.read_csv(shops_path)
test = pd.read_csv(test_path).set_index('ID')

print('items')
print(display(items));
print('categories')
print(display(item_categories));
print('sales')
print(display(sales));
print('shops')
print(display(shops));
print('test')
print(display(test));

items


,item_id,item_name_translated
0,0,!! IN THE POWER OF HAPPINESS (PLAST) D
1,1,! ABBYY FineReader 12 Professional Edition Ful...
2,2,*** IN THE GLORY OF THE GLORY (UNV) D
3,3,*** BLUE WAVE (Univ) D
4,4,*** BOX (GLASS) D
...,...,...
22165,22165,"Nuclear Titbit 2 [PC, Digital Version]"
22166,22166,Query language 1C: Enterprise [Digital version]
22167,22167,The query language is 1C: Enterprise 8 (+ CD)....
22168,22168,Egg for Little Inu


None
categories


,item_category_id,item_category_name_translated
0,0,PC - Headsets / Headphones
1,1,Accessories - PS2
2,2,Accessories - PS3
3,3,Accessories - PS4
4,4,Accessories - PSP
...,...,...
79,79,Office
80,80,Official - Tickets
81,81,Clean carriers (spire)
82,82,Clean media (piece)


None
sales


...

None
shops


,shop_id,shop_name_translated
0,0,"Yakutsk Ordzhonikidze, 56 francs"
1,1,"Yakutsk TC ""Central"" franc"
2,2,"Adygea Shopping Center ""Mega"""
3,3,"Balashikha TRK ""October-Kinomir"""
4,4,"Volzhsky shopping center ""Volga Mall"""
5,5,"Vologda Shopping and Entertainment Center ""Mar..."
6,6,"Voronezh (Plekhanovskaya, 13)"
7,7,"Voronezh TRC ""Maksimir"""
8,8,"Voronezh TRC City-Park ""Grad"""
9,9,Outbound Trade


None
test


,shop_id,item_id
ID,,
0,5,5037
1,5,5320
2,5,5233
3,5,5232
4,5,5268
...,...,...
214195,45,18454
214196,45,16188
214197,45,15757


None


In [ ]:
# Create month, year and day of week features
sales['date'] = pd.to_datetime(sales['date'], format = '%d.%m.%Y')
sales['month'] = sales['date'].dt.month
sales['year'] = sales['date'].dt.year
sales['day_of_week'] = sales['date'].dt.day_of_week
sales = sales.drop('date',axis=1)
display(sales)

,date_block_num,shop_id,item_id,item_price,item_cnt_day,month,year,day_of_week
0,0,59,22154,999.00,1.0,1,2013,2
1,0,25,2552,899.00,1.0,1,2013,3
2,0,25,2552,899.00,-1.0,1,2013,5
3,0,25,2554,1709.05,1.0,1,2013,6
4,0,25,2555,1099.00,1.0,1,2013,1
...,...,...,...,...,...,...,...,...
2935844,33,25,7409,299.00,1.0,10,2015,5
2935845,33,25,7460,299.00,1.0,10,2015,4
2935846,33,25,7459,349.00,1.0,10,2015,2
2935847,33,25,7440,299.00,1.0,10,2015,3


In [ ]:
def cross_validation_time_series(sales):
    cv = SlidingWindowSplitter(window_length=5, fh=[1, 2, 3], initial_window=10, step_length=3)

    train_windows = []
    test_windows = []
    for i, (train, test) in enumerate(cv.split(sales)):
        np.array(train_windows.append(train))
        np.array(test_windows.append(test))
    return train_windows, test_windows

In [ ]:
#sales[sales['date_block_num'] == 33]
#sales[sales['date_block_num'] < 33]
sales = (sales - sales.mean()) / sales.std()
y = sales['item_cnt_day']
x = sales.drop('item_cnt_day',axis=1)
    
fold = 5      
train_acc_scores = []
rf = RandomForestClassifier()
regressor = make_pipeline(TSFreshFeatureExtractor(show_warnings=False, disable_progressbar=True),rf)
forecaster = make_reduction( regressor, scitype="time-series-regressor", window_length=12)

x_rem, x_test, y_rem, y_test = temporal_train_test_split(x[:1000],y[:1000], test_size=0.15)
sales_rem = np.column_stack((x_rem,y_rem))
sales_rem = pd.DataFrame(sales_rem, columns=sales.columns)
tscv = TimeSeriesSplit(gap=0, max_train_size=None, n_splits=fold, test_size=None)
loop= tqdm(sales_rem,leave=True)

for i in range(fold):
    for train_index, val_index in tscv.split(sales_rem):
        y_rem = sales_rem['item_cnt_day']
        x_rem = sales_rem.drop('item_cnt_day',axis=1)
        display(y_rem)
        #print("TRAIN:", train_index, "Validation:", val_index)
        x_train, x_val = x_rem.iloc[train_index], x_rem.loc[val_index]
        y_train, y_val = y_rem.iloc[train_index], y_rem.loc[val_index]
        
        display(x_train)
        forecaster.fit(x_train,y_train)
        fh = ForecastingHorizon(x_train.index, is_relative=False)
        train_pred = forecaster.predict(fh)
        train_acc_score =accuracy_score(y_train, train_pred)
        train_acc_scores.append(train_acc_score)
        
        fold +=1
        loop.set_description(f"Fold [{i}/{fold}]")
        loop.set_postfix(training_accuracy= f"{training_accuracy}")

    print("Train size: %", "%.3f" % ((len(x_train)/len(x))*100),f"({len(x_train)} samples)")
    print("Test size: %", "%.3f" % ((len(x_test)/len(x))*100),f"({len(x_test)} samples)")
    print("Validation size: %", "%.3f" % ((len(x_val)/len(x))*100),f"({len(x_val)} samples)")

    print(np.mean(train_acc_scores))

In [ ]:
# y_train, y_test = temporal_train_test_split(sales[:10000])

# regressor = make_pipeline(
#     TSFreshFeatureExtractor(show_warnings=False, disable_progressbar=True),
#     RandomForestRegressor(),
# )
# forecaster = make_reduction(
#     regressor, scitype="time-series-regressor", window_length=12
# )
# forecaster.fit(y_train)

# fh = ForecastingHorizon(y_test.index, is_relative=False)
# y_pred = forecaster.predict(fh)